In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('google_account.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
temp = client.open("rolands budgetvariant")


In [3]:
print(temp)
dir(temp)
print(temp.worksheets())
sheet = temp.worksheet("ALLES")

<Spreadsheet 'rolands budgetvariant' id:1id3-2Gcwh4ENskVG6pVTTkTeKHjWcqExfsQ7Oh4hIO4>
[<Worksheet 'Sheet1' id:od6>, <Worksheet 'november 2017' id:owor8xu>, <Worksheet 'ALLES' id:ov01ozr>, <Worksheet 'NL94INGB0703873202_05-12-2014_09-12-2017' id:oy9fdfi>]


In [5]:
import nltk
def get_freq_dist_for_sheet(sheet, key, max_words=30):
    records = sheet.get_all_records()
    words = list()
    for record in records:
        words.extend(w.lower() for w in record[key].split())
    
    all_words = nltk.FreqDist(words)
    word_features = list(all_words)[:max_words]
    return word_features

interesting_features = ["Naam / Omschrijving", "Tegenrekening", "Af Bij", "Mededelingen"]
freq_dists = dict()
for feature in interesting_features:
    freq_dists[feature] = get_freq_dist_for_sheet(sheet, feature)


In [7]:
def record_features(record, key, doc_features): 
    document_words = set(w.lower() for w in record[key].split()) 
    features = {}
    for word in doc_features:
        features['contains({},{})'.format(key,word)] = (word in document_words)
    return features

def all_record_features(record):
    input_data = dict()
    for categorie_name in freq_dists:
        ## dict.update means that you merge dictionaries
        input_data.update(record_features(record, categorie_name, freq_dists[categorie_name]))
    return input_data

def get_traindata(sheet):
    records = sheet.get_all_records()
    traindata = list()
    for record in records:
        if record["Categorie"]:
            input_data = all_record_features(record)
            traindata.append((input_data, record["Categorie"]))
    return traindata


In [8]:
client = gspread.authorize(creds)
temp = client.open("rolands budgetvariant")
sheet = temp.worksheet("ALLES")

a = get_traindata(sheet)
all_labels = set([x[1] for x in a])
print(all_labels)


{'parkeren', 'electriciteit', 'internet', 'huur', 'water', 'wegenbelasting', 'verzekering auto', 'inleg lisette', 'overig', 'openbaar vervoer', 'abonnementen', 'benzine', 'boodschappen', 'reizen', 'goede doelen', 'zorgverzekering roland', 'waterschapsbelasting', 'tanken', 'sport', 'uit eten', 'cash', 'verzekering roland', 'inleg roland', 'sport lisette', 'auto'}


In [32]:
                 
print("Training with " + str(len(a)) + " entries")
#classifier = nltk.NaiveBayesClassifier.train(a)
# classifier = nltk.classify.DecisionTreeClassifier.train(a)
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
from sklearn import tree

classifier = SklearnClassifier(tree.DecisionTreeClassifier(), sparse=False).train(a)

print(classifier)
records = sheet.get_all_records()

for row, record in enumerate(records):
    try:
        row += 2 # rows start at 1... first row is a header
        input_data = all_record_features(record)
        but = classifier.classify(input_data)
        if but != record["Computer guessed"]:
            sheet.update_cell(row, 11, but)
    except Exception as e:
        print("Exception at row " + str(row))

Training with 365 entries
<SklearnClassifier(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))>


RequestError: (401, '401: b\'<HTML>\\n<HEAD>\\n<TITLE>Unauthorized</TITLE>\\n</HEAD>\\n<BODY BGCOLOR="#FFFFFF" TEXT="#000000">\\n<H1>Unauthorized</H1>\\n<H2>Error 401</H2>\\n</BODY>\\n</HTML>\\n\'')

# Print that stuff!

In [37]:
import graphviz 

dot_data = tree.export_graphviz(classifier._clf, out_file=None, 
                         feature_names=classifier._vectorizer.get_feature_names(),  
                         class_names=classifier.labels(),  
                         filled=True, rounded=True,  
                         special_characters=True)
graph = graphviz.Source(dot_data) 
graph.render("budget_decisiontree") 


'budget_decisiontree.pdf'

In [24]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

classif = SklearnClassifier(BernoulliNB()).train(a)
for row, record in enumerate(records):
    try:
        row += 2 # rows start at 1... first row is a header
        input_data = all_record_features(record)
        but = classifier.classify(input_data)
        if but != record["Computer guessed"]:
            print("Changing " + str(record) + " to: " + but)
            sheet.update_cell(row, 11, but)
    except Exception as e:
        print("Exception at row " + str(row))

Changing {'Datum': 20171023, 'Naam / Omschrijving': 'Belastingdienst', 'Rekening': 'NL94INGB0703873202', 'Tegenrekening': 'NL86INGB0002445588', 'Code': 'GT', 'Af Bij': 'Af', 'Bedrag (EUR)': '74,00', 'MutatieSoort': 'Online bankieren', 'Mededelingen': 'Naam: Belastingdienst Omschrijving: 7204002487870004 IBAN: NL86INGB0002445588', 'Categorie': 'auto', 'Computer guessed': 'auto'} to: wegenbelasting
Changing {'Datum': 20170807, 'Naam / Omschrijving': 'MW K J F Boere', 'Rekening': 'NL94INGB0703873202', 'Tegenrekening': 'NL45INGB0009039222', 'Code': 'GT', 'Af Bij': 'Bij', 'Bedrag (EUR)': '10,00', 'MutatieSoort': 'Online bankieren', 'Mededelingen': 'Naam: MW K J F Boere Omschrijving: buurt bbq brouwersgracht IBAN: NL45INGB0009039222', 'Categorie': 'uit eten', 'Computer guessed': 'boodschappen'} to: uit eten
Changing {'Datum': 20170717, 'Naam / Omschrijving': 'Belastingdienst', 'Rekening': 'NL94INGB0703873202', 'Tegenrekening': 'NL86INGB0002445588', 'Code': 'GT', 'Af Bij': 'Af', 'Bedrag (EUR)

In [14]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion="gini")
Xtrain = [x[0] for x in a]
Ytrain = [x[1] for x in a]
tree.fit(Xtrain, Ytrain)

TypeError: float() argument must be a string or a number, not 'dict'

In [71]:
records[0]

{'Af Bij': 'Af',
 'Bedrag (EUR)': '11,36',
 'Categorie': 'boodschappen',
 'Code': 'BA',
 'Computer guessed': '',
 'Datum': 20171208,
 'Mededelingen': 'Pasvolgnr:002 07-12-2017 16:49 Transactie:4135Q2 Term:945N9N',
 'MutatieSoort': 'Betaalautomaat',
 'Naam / Omschrijving': 'Albert Heijn 1520 VEENENDAAL NLD',
 'Rekening': 'NL94INGB0703873202',
 'Tegenrekening': ''}

In [ ]:
records[0].update

In [ ]:

# Extract and print all of the values
records = sheet.get_all_records()
traindata = list()

def get_interesting_information(record):
    return (record["Naam / Omschrijving"], record["Categorie"])
for record in records:
    if record["Categorie"]:
        traindata.append(get_interesting_information(record))
#     print(record)
print(len(traindata))

In [ ]:
import random
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)